<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background- padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       4D Analytics using the New York City Taxi dataset --Geospatial & Visualizations
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>
Businesses that involve different locations and moving things from place to place where they are needed can benefit from geospatial analysis.  Questions include capacity at given locations and capacity to move things from where they are to where they are needed.  A common optimization question is keeping the assets used in movement in constant use.  A major railroad saved millions of dollars using Vantage to reduce the idle time of engines waiting in the rail yard.  A commonly used set of data for this type of analysis is the NYC Taxi data which has trips with cabs (medallions), pickup/drop-off time and geospatial coordinates. , passenger count and fares. Using geospatial analysis, decisions can be made about deploying cabs and vehicles with different capacities.<br>Vantage provides support for geospatial shapes (points, lines, curves, polygon, etc) and methods for analyzing the relationships of those shapes ( Distance, length, overlaps, contains, touches, etc).  In this demo, we’ll be using:</p>
    <ul style = 'font-size:16px;font-family:Arial'>
<li>Geospatial using the ST_GEOMETRY data type and ST_SphericalDistance method</li>
        <li>The visualizations are done using Python plotly module</li>
    </ul>
</p>


<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Standard Libraries
import getpass
import warnings

# Data Manipulation and Visualization Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Teradata Libraries
from teradataml import *
from teradatasqlalchemy.types import *

# Configuration and Display Settings
display.max_rows = 5

# Miscellaneous
%matplotlib inline

# Suppress Warnings
warnings.filterwarnings('ignore')

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=NYC_Taxi_Geospatial_PY_SQL.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys.</p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'> <b>1.1 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_NYCTaxi_cloud');"    # Takes about 2 minutes
%run -i ../run_procedure.py "call get_data('DEMO_NYCTaxi_local');"    # Takes about 3 minutes

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'> <b>1.2 Access data in Vantage  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>Let us check the data sample. This demonstration will use two tables: the taxi trip details and the fares for each trip. The queries below will sample each table and then show the range of the time period covered by the data. </p>

In [ ]:
DataFrame(in_schema("DEMO_NYCTaxi", "Trip"))

In [ ]:
DataFrame(in_schema("DEMO_NYCTaxi", "Trip_Fare"))

In [ ]:
qry = """ 
SELECT MIN(pickup_datetime) as min_pickup, MAX(dropoff_datetime) as max_pickup FROM DEMO_NYCTaxi.Trip;
"""

DataFrame.from_query(qry)

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>2. Geospatial Analysis </b></p>
<p style = 'font-size:16px;font-family:Arial'> Now we have seen the trip and fare details, Let's define a few landmarks. </p>

In [ ]:
qry = """ 
CREATE MULTISET TABLE dim_geo_locations (
    location VARCHAR(100),
    Lat FLOAT,
    Lon FLOAT,
    geo_point SYSUDTLIB.ST_GEOMETRY(16776192) INLINE LENGTH 9920
) PRIMARY INDEX (location);
"""
try:
    execute_sql(qry)
except:
    db_drop_table("dim_geo_locations")
    execute_sql(qry)

In [ ]:
qry = """ 
INSERT INTO dim_geo_locations VALUES ('Columbia University', 40.81, -73.96, 'POINT(40.81 -73.96)');
INSERT INTO dim_geo_locations VALUES ('Empire State Building', 40.75, -73.99, 'POINT(40.75 -73.99)');
INSERT INTO dim_geo_locations VALUES ('Grand Central Station', 40.75, -73.98, 'POINT(40.75 -73.98)');
INSERT INTO dim_geo_locations VALUES ('JFK Airport', 40.64, -73.79, 'POINT(40.64 -73.79)');
INSERT INTO dim_geo_locations VALUES ('Madison Square Garden', 40.75, -73.99, 'POINT(40.75 -73.99)');
INSERT INTO dim_geo_locations VALUES ('New York Stock Exchange', 40.71, -74.01, 'POINT(40.71 -74.01)');
INSERT INTO dim_geo_locations VALUES ('Times Square', 40.76, -73.99, 'POINT(40.76 -73.99)');
INSERT INTO dim_geo_locations VALUES ('United Nations HQ', 40.75, -73.97, 'POINT(40.75 -73.97)');
INSERT INTO dim_geo_locations VALUES ('Yankee Stadium', 40.83, -73.93, 'POINT(40.83 -73.93)');
"""

out = execute_sql(qry)

In [ ]:
DataFrame("dim_geo_locations")

<p style = 'font-size:16px;font-family:Arial'> As you can see the dim_geo_locations contain a separate Latitude and Longitude column and a "Well Known Text" (WKT) geospatial column with a POINT defined. The supported shape types also have user defined type. Those available are:
<ul style = 'font-size:16px;font-family:Arial'>
    <li>ST_CircularString</li>
    <li>ST_GeomCollection</li>
    <li>ST_MultiLineString</li>
    <li>ST_Point</li>
    <li>ST_CompoundCurve</li>
    <li>ST_Geometry</li>
    <li>ST_MultiPoint</li>
    <li>ST_Polygon</li>
    <li>ST_Curve</li>
    <li>ST_LineString</li>
    <li>ST_MultiPolygon</li>
    <li>ST_Surface</li>
    <li>ST_CurvePolygon</li>
    <li>ST_MultiCurve</li>
    <li>ST_MultiSurface</li></ul>


<p style = 'font-size:16px;font-family:Arial'> Now let's plot these locations on the map.

In [ ]:
# Dim geo locations
geo = (
    DataFrame.from_query("SELECT location, Lat, Lon FROM dim_geo_locations;")
    .to_pandas()
    .reset_index()
)
fig1 = px.scatter_mapbox(
    geo,
    lat="Lat",
    lon="Lon",
    hover_name="location",
    color_discrete_sequence=["red"],
    zoom=11,
    height=400,
)
fig1.update_layout(mapbox_style="open-street-map")
fig1.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig1.show()

<p style = 'font-size:16px;font-family:Arial'>
Sometimes the source data you get from other sources may not have been developed with geospatial data types but separate latitude and longitude as seen in Trips table above. Ideally data would be stored with geospatial data types but we can cast them as geometry datatype. To simplify the code, we will create a function that will create a WKT representation of the point

In [ ]:
qry = """
REPLACE FUNCTION Make_Geometry(
    latitude FLOAT,
    longitude FLOAT
) 
RETURNS ST_GEOMETRY
LANGUAGE SQL CONTAINS SQL COLLATION INVOKER INLINE TYPE 1
RETURN CAST('POINT(' || TRIM(latitude (DECIMAL(15,6))) || ' ' || TRIM(longitude (DECIMAL(15,6))) || ')' AS ST_GEOMETRY);
"""

out = execute_sql(qry)


 <p style = 'font-size:16px;font-family:Arial'>
    Here are the coordinates for the JFK airport, and this is a point:

In [ ]:
qry = """
SELECT Make_Geometry(40.64, -73.79) AS geometry;
"""

DataFrame.from_query(qry)

<p style = 'font-size:16px;font-family:Arial'> Let's find out the drop locations for taxis staring from JFK Airport.

In [ ]:
qry = """
SELECT
    r.dropoff_latitude AS dropoff_latitude,
    r.dropoff_longitude AS dropoff_longitude,
    COUNT(*) AS "Count"
FROM
    DEMO_NYCTaxi.Trip r
WHERE
    CAST(pickup_latitude AS DECIMAL(10,2)) = 40.64
    AND CAST(pickup_longitude AS DECIMAL(10,2)) = -73.79
GROUP BY
    1, 2;
"""

df = DataFrame.from_query(qry)
df

<p style = 'font-size:16px;font-family:Arial'> Let's see these drop points on the map.

In [ ]:
drop_loc = df.to_pandas()

fig2 = px.scatter_mapbox(
    drop_loc,
    lat="dropoff_latitude",
    lon="dropoff_longitude",
    color_discrete_sequence=["blue"],
    zoom=11,
    height=400,
)
fig2.update_layout(mapbox_style="open-street-map")
fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig2.show()

<p style = 'font-size:16px;font-family:Arial'> Let's visualize these drop locations with the landmark locations we have created before.

In [ ]:
fig3 = go.Figure(data=fig1.data + fig2.data, layout=fig1.layout)
fig3.show()

<p style = 'font-size:16px;font-family:Arial'> Vantage has many inbuilt geospatial functions e.g ST_Spherical_Distance which calculates the distance between two points. For complete list of geospatial functions you can check the reference link.

<p style = 'font-size:16px;font-family:Arial'> Now let's filter the rides staring from JFK Airport where drop off point is within 0.5km of any landmark

In [ ]:
qry = """
SELECT
    geoloc.location AS location,
    trip.dropoff_latitude AS dropoff_latitude,
    trip.dropoff_longitude AS dropoff_longitude
FROM
    DEMO_NYCTaxi.trip trip
JOIN
    dim_geo_locations geoloc
ON
    geoloc.geo_point.ST_SphericalDistance(make_geometry(trip.dropoff_latitude, trip.dropoff_longitude)) < 500
WHERE
    CAST(pickup_latitude AS DECIMAL(10,2)) = 40.64
    AND CAST(pickup_longitude AS DECIMAL(10,2)) = -73.79
GROUP BY
    1, 2, 3;
"""

df = DataFrame.from_query(qry)
df

In [ ]:
drop_loc2 = df.to_pandas()

fig4 = px.scatter_mapbox(
    drop_loc2,
    lat="dropoff_latitude",
    lon="dropoff_longitude",
    hover_name="location",
    color_discrete_sequence=["blue"],
    zoom=11,
    height=400,
)
fig4.update_layout(mapbox_style="open-street-map")
fig4.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig5 = go.Figure(data=fig1.data + fig4.data, layout=fig1.layout)
fig5.show()

<p style = 'font-size:16px;font-family:Arial'> Which landmark has the highest number of pickup points within 0.5KM

In [ ]:
qry = """
SELECT
    geoloc.location AS Location,
    geoloc.lat AS Lat,
    geoloc.lon AS Lon,
    COUNT(1) AS pickup_cnt
FROM
    DEMO_NYCTaxi.trip trip
JOIN
    dim_geo_locations geoloc
ON
    geoloc.geo_point.ST_SphericalDistance(make_geometry(trip.pickup_latitude, trip.pickup_longitude)) < 500
GROUP BY
    1, 2, 3;
"""

df = DataFrame.from_query(qry)
df

In [ ]:
pickup = df.to_pandas()

fig6 = px.scatter_mapbox(
    pickup,
    lat="Lat",
    lon="Lon",
    hover_name="Location",
    color="pickup_cnt",
    size="pickup_cnt",
    zoom=11,
    height=400,
)
fig6.update_layout(mapbox_style="open-street-map")
fig6.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig6.show()

<p style = 'font-size:16px;font-family:Arial'> What is the number of pickup at 'JFK Airport' throughout the day? As we have earlier from min & max pickup datetime we have only one day's data hence we don't need any date filter.

In [ ]:
qry = """
SELECT
    $TD_TIMECODE_RANGE AS time_bucket_per_hr,
    l.location AS Location,
    COUNT(1) AS pickup_cnt
FROM
    DEMO_NYCTaxi.Trip r
JOIN
    dim_geo_locations l
ON
    l.geo_point.ST_SphericalDistance(make_geometry(r.pickup_latitude, r.pickup_longitude)) < 1000
GROUP BY
    time(minutes(60) AND l.location)
USING TIMECODE(pickup_datetime)
WHERE
    l.location = 'JFK Airport';
"""

df = DataFrame.from_query(qry).sort(["Location", "time_bucket_per_hr"])
df

In [ ]:
hr_throughput = df.to_pandas()
hr_throughput.plot(kind="bar", legend=True, figsize=(12, 9))

<p style = 'font-size:16px;font-family:Arial'> From the chart above we can see that the pickup demand at airport is higher from 3-10PM and very low from 2-5am.

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>3. Clean up </b></p>
<p style = 'font-size:18px;font-family:Arial'> <b>3.1 Worktables </b></p>

In [ ]:
tables = ["dim_geo_locations", "make_geometry"]

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'> <b>3.2 Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_NYCTaxi');" 
# Takes 10 seconds takes about 10 seconds, optional if you want to use the data later
# Note that the same database & tables are used in Usecases/NYC-taxi-4d/NYC-taxi-4d.ipynb 

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>4. Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>
In this demonstration we have seen Vantage can store common geometry datatype like point, linestring etc in ST_GEOMETRY datatype and has inbuild functions which are fairly simple and easy to use. For more information on the geometry datatype and functions please refer to link below. 

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.2 Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
        <li>Teradata® Geospatial Utilities User Guide: <a href = 'https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide'>https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide</a></ul>

<footer style="padding-bottom:35px; background:#91A0AB; ">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>